# <b>ISIN SCRAPER</b>
======================================================

In [67]:
# Imports
# For webscraping, import the following libraries from selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Exceptions
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
import traceback

# Import datasets

In [56]:
# Read the different datas from the csv files
emmissions_df = pd.read_csv('EmissionsDataISIN.csv')
full_report_df = pd.read_csv('full_report_df_sorted.csv')

In [70]:
# get unique rows based on full_report_df['Company'], and maintain the order of the rows
unique_companies = full_report_df.drop_duplicates(subset='Company', keep='first')

# add column to the unique_companies dataframe
unique_companies['ISIN'] = ""

/var/folders/bt/vkg37vm51db7424mjftzw2700000gn/T/ipykernel_73393/2469016511.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_companies['ISIN'] = ""


# FUNCTIONS

In [12]:
# LOGIN
def login():
    WebDriverWait(driver, 10).until(lambda driver: driver.find_element(By.ID, "user_login"))
    username_field = driver.find_element(By.ID, "user_login").send_keys("2101265@sit.singaporetech.edu.sg")
    password_field = driver.find_element(By.ID, "user_pass").send_keys("President2024")
    
    driver.find_element(By.ID, "user_pass").send_keys(Keys.ENTER)
    
    WebDriverWait(driver, 10).until(lambda driver: driver.find_element(By.ID, "menu-item-71"))
    driver.get('https://www.isin.com/isin-database/')


# 1) LOGIN

In [57]:
# PATHS
ISIN_URL = 'https://www.isin.com/wp-login.php'

# set the options for the chrome driver
options = webdriver.ChromeOptions()
options.set_capability("goog:loggingPrefs", {"browser": "ALL"})

# import the driver 
driver = webdriver.Chrome(options=options)

# access the site 
driver.get(ISIN_URL)

# LOGIN AND ENTER ISIN SEARCH PAGE
login()

# 2) SEARCH

In [84]:
# For each company in the unique_companies dataframe, search for the ISIN
# print(full_report_df)

# loop through the unique_companies dataframe
for index, row in unique_companies.iterrows():
    if row['ISIN'] != '':
        print(f'{index}. Company: {row["Company"]}\nISIN: {row["ISIN"]}\n')
    else:
        # flag for whether type of security is equity
        equity = False

        # locate the <select> element by its name attribute and create a Select object for the located <select> element
        select = Select(driver.find_element(By.NAME, "search_select"))
        # select the option where the value is "title"
        select.select_by_value("title")
            
        name = row['Company']
        search_bar = driver.find_element(By.ID, 'isin').send_keys(name)
        driver.find_element(By.ID, 'isin').send_keys(Keys.ENTER)

        print('========================================')
        
        # if there is a table body, then there are results
        try:
            # if WebDriverWait(driver, 10).until(lambda driver: driver.find_element(By.TAG_NAME, 'tbody')):
            if driver.find_element(By.TAG_NAME, 'tbody'):
                company_rows = driver.find_element(By.XPATH, '//*[@id="free_data_report_dt_"]/tbody')
                companies = company_rows.find_elements(By.TAG_NAME, 'tr')
                for company in companies:
                    if company.find_element(By.XPATH, './/td[5]').text.lower() == 'equity':
                        equity = True
                        # get ISIN
                        isin = company.find_element(By.XPATH, './/td[2]').text
                        print(f'{index}. Company: {company.find_element(By.XPATH, ".//td[1]").text}\nISIN: {isin}')
                        
                        # save that isin in that row's ['ISIN'] column
                        unique_companies.at[index, 'ISIN'] = isin
                
                if not equity:
                    # we dont know what to do yet, so just print
                    print(f'{index}. Company: ', name)
                    print('NO EQUITY TYPE')
                    
                    unique_companies.at[index, 'ISIN'] = 'NO EQT'
                    
                    # # just get the first result
                    # isin = companies[0].find_element(By.XPATH, './/td[2]').text
                    # print(f'{index}. Company: {companies[0].find_element(By.XPATH, ".//td[1]").text}\nISIN: {isin}')
                    
                    # # save that isin in that row's ['ISIN'] column
                    # unique_companies.at[index, 'ISIN'] = isin
        except:
            try:
                # no results found
                if WebDriverWait(driver, 10).until(lambda driver: driver.find_element(By.XPATH, '//p[contains(text(), "No Results Found.")]')):
                    print(f'{index}. Company: ', name)
                    print('No ISIN found')
                    unique_companies.at[index, 'ISIN'] = 'NIL'
            except:
                # print the call stack/traceback
                print(traceback.format_exc())
                
        print('========================================\n')


0. Company: ABRAXAS PETROLEUM CORP
ISIN: US0038301067

1. Company: ACTIVE POWER INC
ISIN: US00504W3088

2. Company: ADAMS RESOURCES & ENERGY, INC.
ISIN: NIL

3. Company: ADAPTEC INC
ISIN: NIL

4. Company: AES CORP
ISIN: US00130H1059

5. Company: AGL RESOURCES INC
ISIN: US0012041069

6. Company: ALLEGHENY ENERGY, INC
ISIN: NIL

7. Company: ALLETE INC
ISIN: US0185221027

8. Company: ALLIANCE RESOURCE PARTNERS LP
ISIN: US01877R1086

9. Company: ALLIANT ENERGY CORP
ISIN: US0188021085

10. Company: ALLIS CHALMERS ENERGY INC.
ISIN: NIL

11. Company: ALON USA ENERGY, INC.
ISIN: NIL

12. Company:  ALPHA NATURAL RESOURCES, INC.
No ISIN found

13. Company: AMEREN CORP
ISIN: US0236082014

14. Company:  AMERICAN DG ENERGY INC
No ISIN found

15. Company:  AMERICAN ELECTRIC POWER CO INC
No ISIN found

16. Company:  AMERICAN NATURAL ENERGY CORP
No ISIN found

17. Company:  AMERICAN OIL & GAS INC
No ISIN found

18. Company: ANADARKO PETROLEUM CORP
ISIN: US0325111070

19. Company: APACHE CORP
ISIN: US0

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="search_select"]"}
  (Session info: chrome=126.0.6478.182); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000100f7ea0c chromedriver + 4385292
1   chromedriver                        0x0000000100f77318 chromedriver + 4354840
2   chromedriver                        0x0000000100b94b0c chromedriver + 281356
3   chromedriver                        0x0000000100bd72f8 chromedriver + 553720
4   chromedriver                        0x0000000100c0fd24 chromedriver + 785700
5   chromedriver                        0x0000000100bcbeec chromedriver + 507628
6   chromedriver                        0x0000000100bcc8c4 chromedriver + 510148
7   chromedriver                        0x0000000100f463c8 chromedriver + 4154312
8   chromedri

In [87]:
# get first 4138 rows from the unique_companies dataframe
data = unique_companies.head(3840)

In [95]:
# get data
# count the number of rows with 'ISIN' column where the value is not 'NIL'
non_nil = data[data['ISIN'] != 'NIL'].count()
nil = data[data['ISIN'] == 'NIL'].count()
print(f'Total number of rows with ISIN: {non_nil["ISIN"]} / {len(data)} = {non_nil["ISIN"]/len(data)*100}%')
print(f'Total number of rows without ISIN: {nil["ISIN"]} / {len(data)} = {nil["ISIN"]/len(data)*100}%')

Total number of rows with ISIN: 1478 / 3840 = 38.489583333333336%
Total number of rows without ISIN: 2362 / 3840 = 61.510416666666664%
